# Color Matching Image Style Transfer
CNNs tend to be sensitive to changes in input color distributions between training and target domains. Since images created by PD and those of many cameras differ in saturation and color distribution, we want a way of aligning those. Style transfer methods like GANs often hallucinate artifacts or add high frequency noise. We want to have an approach of aligning color spaces in a deterministic way, that won't change to content.

The Image Style Transfer technique we reference to as Color Matching is based of [this Paper by Erik Reinhard](https://www.cs.tau.ac.il/~turkel/imagepapers/ColorTransfer.pdf). It approximates the color distributions of the source and target domain using channel wise gaussian distribution. It then computes an Affine Transformation between the source and target distributions. Applying the transformation to images transfers them from the source domain to the target domain.

In most cases we want to transfer the synthetic images closer to the real world domain, so the source color distribution is the distribution of the synthetic images, the target distribution is the color distribution of th real camera images.

| | original | transformed |
|---|---|---|
| PD Image | ![PD Highway](../../_static/color_matcher_pd_highway.png) |   |
| Image from [Cityscapes](https://ieeexplore.ieee.org/document/7780719) | ![Cityscapes](../../_static/color_matcher_cityscapes.png) | ![PD Cityscapes](../../_static/color_matcher_pd_highway_cityscapes.png) |
| Image from [Nuscenes](https://arxiv.org/abs/1903.11027) | ![Nuscenes](../../_static/color_matcher_nuscenes.jpeg) | ![PD Nuscenes](../../_static/color_matcher_pd_highway_nuscenes.png) |

## Usage

The main factor determining the quality of results is using representative color statistics. The usage therefore often has two steps: Calculation of the statistics and applying the statistics to the whole dataset. You can also calculate the color distribution on your data:

In [ ]:
from paralleldomain.utilities.color_matcher import ColorMatcher, GaussianColorDistribution

source_distribution = GaussianColorDistribution.from_folder(image_folder="path/to/a/folder/with/images")
target_distribution = GaussianColorDistribution.from_dataset_path(dataset_path="path/to/an/dgp/dataset", dataset_format="dgp")
# You can also create it using a custom image iterator
iterator_distribution = GaussianColorDistribution.from_image_stream(image_stream=iter(some_image_loader))

The color transformation can be calculated between a source and a target distribution:

In [ ]:
matcher = ColorMatcher.from_distributions(source=source_distribution, target=target_distribution)

Both the distributions and the transform can be stored to json and loaded again:

In [ ]:
source_distribution.save_to_json("some/json/path.json")
loaded_distribution = GaussianColorDistribution.from_json("some/json/path.json")
# Same for the matcher:
matcher.save_to_json("some/json/path.json")
loaded_matcher = ColorMatcher.from_json("some/json/path.json")

To apply the color matcher to an image simply use the matmul operator:

In [ ]:
color_matched = matcher @ image

## Limitations

We approximate the color shift of the camera by looking at some example images. This only works if the color shift in the scenes is not too large. It's therefore recommended to exclude tunnels and similar from the statistic calculations. It might be necessary to calculate multiple distributions for different times of day.
The method is also not applicable for relighting. Transforming night images to day images or the other way arround shows the limitations:

| source | target | transformed |
|---|---|---|
| ![PDNight](../../_static/color_matcher_pd_night.png) | ![Cityscapes2](../../_static/color_matcher_cityscapes.png) | ![CityscapesNight](../../_static/color_matcher_cityscapes_night.png) |
| ![Cityscapes3](../../_static/color_matcher_cityscapes.png) | ![PD Night2](../../_static/color_matcher_pd_night.png) | ![PDNightCityscapes](../../_static/color_matcher_pd_night_cityscapes.png) |
